---
title: "Assignment 3 - Solutions"
bibliography: ../reading_list.bib
format:
  html:
    toc: true
    toc-depth: 2
    toc-location: left
    theme: default
    code-fold: false
    code-tools: true
    embed-resources: true
    self-contained: true
---

In [ ]:
using DataFrames, DataFramesMeta, CSV, StatsPlots, LinearAlgebra, Distributions, StatsBase

# Question 1: Deriving Variance and Covariance Relationships

Given the income process:
$$ y_{it} = \mu_{t} + \alpha_{i} + \zeta_{it} + \epsilon_{it} $$

where $\zeta_{it} = \rho \zeta_{it-1} + \eta_{it}$, $\alpha_{i} \sim (0, \sigma^2_{\alpha})$, $\epsilon_{it} \sim N(0, \sigma^2_{\epsilon})$, and $\eta_{it} \sim N(0, \sigma^2_{\eta})$ are all independent.

**Variance of $y_{it}$:**

Since all components are independent:
$$
\begin{align}
\mathbb{V}[y_{it}] &= \mathbb{V}[\mu_t + \alpha_i + \zeta_{it} + \epsilon_{it}] \\
&= \mathbb{V}[\alpha_i] + \mathbb{V}[\zeta_{it}] + \mathbb{V}[\epsilon_{it}] \\
&= \sigma^2_{\alpha} + \mathbb{V}[\zeta_{it}] + \sigma^2_{\epsilon}
\end{align}
$$

**Covariance of $y_{it}$ and $y_{it+s}$:**

For the same individual $i$ at different times:
$$
\begin{align}
\mathbb{C}(y_{it}, y_{it+s}) &= \mathbb{C}(\alpha_i + \zeta_{it} + \epsilon_{it}, \alpha_i + \zeta_{it+s} + \epsilon_{it+s}) \\
&= \mathbb{C}(\alpha_i, \alpha_i) + \mathbb{C}(\zeta_{it}, \zeta_{it+s}) + \mathbb{C}(\epsilon_{it}, \epsilon_{it+s})
\end{align}
$$

Since $\alpha_i$ is permanent: $\mathbb{C}(\alpha_i, \alpha_i) = \mathbb{V}[\alpha_i] = \sigma^2_{\alpha}$

Since $\epsilon_{it}$ is measurement error (iid): $\mathbb{C}(\epsilon_{it}, \epsilon_{it+s}) = 0$ for $s > 0$

For the AR(1) component: $\mathbb{C}(\zeta_{it}, \zeta_{it+s}) = \rho^s \mathbb{V}[\zeta_{it}]$

Therefore:
$$
\mathbb{C}(y_{it}, y_{it+s}) = \sigma^2_{\alpha} + \rho^s \mathbb{V}[\zeta_{it}]
$$

# Question 2: Expressing Parameters in Terms of Moments

From Question 1, we have:
$$
\begin{align}
\mathbb{V}[y_{it}] &= \sigma^2_{\alpha} + \mathbb{V}[\zeta_{it}] + \sigma^2_{\epsilon} \\
\mathbb{C}(y_{it}, y_{it+s}) &= \sigma^2_{\alpha} + \rho^s \mathbb{V}[\zeta_{it}]
\end{align}
$$

**Step 1: Solve for $\rho$**

Taking differences of covariances at different lags:
$$
\begin{align}
\mathbb{C}(y_{it}, y_{it+s+1}) - \mathbb{C}(y_{it}, y_{it+s}) &= \rho^{s+1}\mathbb{V}[\zeta_{it}] - \rho^s\mathbb{V}[\zeta_{it}] \\
&= \rho^s(\rho - 1)\mathbb{V}[\zeta_{it}]
\end{align}
$$

Taking the ratio of consecutive differences:
$$
\frac{\mathbb{C}(y_{it}, y_{it+s+2}) - \mathbb{C}(y_{it}, y_{it+s+1})}{\mathbb{C}(y_{it}, y_{it+s+1}) - \mathbb{C}(y_{it}, y_{it+s})} = \frac{\rho^{s+1}(\rho - 1)\mathbb{V}[\zeta_{it}]}{\rho^s(\rho - 1)\mathbb{V}[\zeta_{it}]} = \rho
$$

Therefore:
$$
\rho = \frac{\mathbb{C}(y_{it}, y_{it+3}) - \mathbb{C}(y_{it}, y_{it+2})}{\mathbb{C}(y_{it}, y_{it+2}) - \mathbb{C}(y_{it}, y_{it+1})}
$$

**Step 2: Solve for $\mathbb{V}[\zeta_{it}]$**

Once we have $\rho$, we can use:
$$
\mathbb{V}[\zeta_{it}] = \frac{\mathbb{C}(y_{it}, y_{it+1}) - \sigma^2_{\alpha}}{\rho}
$$

But this requires $\sigma^2_{\alpha}$, so we first need another approach. Using:
$$
\mathbb{C}(y_{it}, y_{it+2}) - \mathbb{C}(y_{it}, y_{it+1}) = (\rho^2 - \rho)\mathbb{V}[\zeta_{it}]
$$

Therefore:
$$
\mathbb{V}[\zeta_{it}] = \frac{\mathbb{C}(y_{it}, y_{it+2}) - \mathbb{C}(y_{it}, y_{it+1})}{\rho^2 - \rho} = \frac{\mathbb{C}(y_{it}, y_{it+2}) - \mathbb{C}(y_{it}, y_{it+1})}{\rho(\rho - 1)}
$$

**Step 3: Solve for $\sigma^2_{\alpha}$**

From the covariance formula:
$$
\sigma^2_{\alpha} = \mathbb{C}(y_{it}, y_{it+1}) - \rho \mathbb{V}[\zeta_{it}]
$$

**Step 4: Solve for $\sigma^2_{\epsilon}$**

From the variance formula:
$$
\sigma^2_{\epsilon} = \mathbb{V}[y_{it}] - \sigma^2_{\alpha} - \mathbb{V}[\zeta_{it}]
$$

**Step 5: Solve for $\sigma^2_{\eta}$**


Since we assumed in this model that the initial value of $\zeta_{i0}$ is drawn from the stationary distribution, we can use the AR(1) stationary variance formula:
$$
\mathbb{V}[\zeta_{it}] = \frac{\sigma^2_{\eta}}{1 - \rho^2}
$$

Therefore:
$$
\sigma^2_{\eta} = (1 - \rho^2)\mathbb{V}[\zeta_{it}]
$$

Alternatively we could use that:
$$
\mathbb{V}[y_{it+1}] = \sigma^2_{\alpha} + \rho^2\mathbb{V}[\zeta_{it}] + \sigma^2_{\eta} + \sigma^2_{\epsilon}.
$$




# Question 3: Estimating Parameters from PSID Data


In [ ]:
# Load the data
d = CSV.read("../data/abb_aea_data.csv", DataFrame)
d[!, :logy] = log.(d.y);

First, we need to account for the fact that PSID is biennial (every 2 years). This means we observe income at $t$, $t+2$, $t+4$, $t+6$, etc.


In [ ]:
# Construct forward lags (accounting for biennial data)
function forward_lag(d, l, var)
    d_lag = @chain d begin
        @select :person :year $var
        @transform :year = :year .- l
        @rename $(Symbol(var, "_", l)) = $var
    end
    d = leftjoin(d, d_lag, on=[:person, :year])
    return d
end

d = forward_lag(d, 2, :logy)
d = forward_lag(d, 4, :logy)
d = forward_lag(d, 6, :logy)

Demean by age to remove the $\mu_t$ component:


In [ ]:
d = @chain d begin
    groupby(:age)
    @transform :eps = :logy .- mean(:logy)
end

d = forward_lag(d, 2, :eps)
d = forward_lag(d, 4, :eps)
d = forward_lag(d, 6, :eps)

Calculate covariances:


In [ ]:
cov_lag = pairwise(cov, eachcol(d[!, [:eps, :eps_2, :eps_4, :eps_6]]),
                   skipmissing = :pairwise)[:, 1]

println("Empirical moments:")
println("Var(y_t):           ", round(cov_lag[1], digits=4))
println("Cov(y_t, y_{t+2}):  ", round(cov_lag[2], digits=4))
println("Cov(y_t, y_{t+4}):  ", round(cov_lag[3], digits=4))
println("Cov(y_t, y_{t+6}):  ", round(cov_lag[4], digits=4))

Now estimate the parameters. Since PSID is biennial, our lags correspond to $s=2, 4, 6$ years. We need to adjust the formulas accordingly:

For biennial data with lags $s = 2, 4, 6$:
$$
\rho^{2} = \frac{\mathbb{C}(y_{it}, y_{it+6}) - \mathbb{C}(y_{it}, y_{it+4})}{\mathbb{C}(y_{it}, y_{it+4}) - \mathbb{C}(y_{it}, y_{it+2})}
$$


In [ ]:
# Estimate ρ (accounting for biennial data - each lag is 2 years)
# The formula gives us ρ^2, so we take the square root
ρ_squared = (cov_lag[4] - cov_lag[3]) / (cov_lag[3] - cov_lag[2])
ρ = sqrt(ρ_squared)

println("\nEstimated ρ: ", round(ρ, digits=4))

Estimate $\mathbb{V}[\zeta_{it}]$:

For biennial data:
$$
\mathbb{V}[\zeta_{it}] = \frac{\mathbb{C}(y_{it}, y_{it+4}) - \mathbb{C}(y_{it}, y_{it+2})}{\rho^4 - \rho^2}
$$


In [ ]:
V_ζ = (cov_lag[3] - cov_lag[2]) / (ρ^4 - ρ^2)

println("Estimated V[ζ]:     ", round(V_ζ, digits=4))

Estimate $\sigma^2_{\alpha}$:


In [ ]:
σ2_α = cov_lag[2] - ρ^2 * V_ζ

println("Estimated σ²_α:     ", round(σ2_α, digits=4))

Estimate $\sigma^2_{\epsilon}$:


In [ ]:
σ2_ε = cov_lag[1] - σ2_α - V_ζ

println("Estimated σ²_ε:     ", round(σ2_ε, digits=4))

Estimate $\sigma^2_{\eta}$:


In [ ]:
σ2_η = (1 - ρ^2) * V_ζ

println("Estimated σ²_η:     ", round(σ2_η, digits=4))

Calculate the mean income profile:


In [ ]:
var_by_age = @chain d begin
    @subset .!ismissing.(:y)
    groupby(:age)
    @combine begin
        :mean_logy = mean(log.(:y))
        :n = length(:y)
    end
    @subset :n .> 100
end

μ_t = @chain var_by_age begin
    @subset :age .>= 25 :age .< 65
    @orderby :age
    _[!, :mean_logy]
end

Summary of estimated parameters:


In [ ]:
results = DataFrame(
    Parameter = ["ρ", "σ²_ε", "σ²_η", "σ²_α"],
    Estimate = [ρ, σ2_ε, σ2_η, σ2_α]
)

display(results)

Now let's check how well our estimated model fits the empirical autocovariances:


In [ ]:
# Predicted covariances from our model
# Cov(y_t, y_{t+s}) = σ²_α + ρ^s * V[ζ]
lag = [0, 2, 4, 6]
cov_predicted = [σ2_α + ρ^s * V_ζ for s in lag]

# Compare empirical vs predicted
plot(lag, cov_lag,
    label="Empirical",
    marker=:circle,
    markersize=6,
    linewidth=2,
    xlabel="Lag (years)",
    ylabel="Cov(log yₜ, log yₜ₊ₛ)",
    title="Model Fit: Autocovariance Function",
    legend=:topright)
plot!(lag, cov_predicted,
    label="Model (AR1 + Fixed Effect)",
    marker=:square,
    markersize=6,
    linewidth=2,
    linestyle=:dash)

The plot shows how well the model with fixed effects and AR(1) component fits the empirical autocovariance structure. Notice that the fixed effect $\alpha_i$ creates a floor in the autocovariance function (at $\sigma^2_\alpha$), while the AR(1) component decays according to $\rho^s$.

# Question 4: Comparing ρ Estimates

In [Recitation 4](../recitations/recitation-4.qmd), we estimated an AR(1) process without a permanent component and without measurement error:
$$ y_{it} = \mu_t + \varepsilon_{it}, \quad \varepsilon_{t+1} = \rho \varepsilon_t + \zeta_{t+1} $$

The estimate was approximately $\rho \approx 0.82$, indicating lower persistence. 

Notice that $\rho$ is identified by the rate of decay in the autocovariance *beyond* the first lag. There is a large drop between lags of 0 and 2 in the data, which the richer model can attribute to measurement error. The simpler model must fit this rate of decay by understating the persistence of $\zeta_{it}$.


# Question 5 (Extra Credit): Welfare Analysis with New Income Process

We now repeat the welfare analysis from Recitation 4 using the new income process that includes permanent heterogeneity, AR(1) dynamics, and measurement error.


In [ ]:
# Utility function
utility(c, σ) = c^(1 - σ) / (1 - σ)

# Tauchen method (from recitation)
function tauchen(N, ρ, σ)
    σ_y = σ / sqrt(1 - ρ^2)
    y_max = 3. * σ_y
    y_min = -3. * σ_y
    grid = range(y_min, y_max, length=N)
    step = (y_max - y_min) / (N - 1)
    Π = zeros(N, N)
    d = Normal(0, 1)

    for i in 1:N
        for j in 1:N
            if j == 1
                Π[j, i] = cdf(d, (grid[j] - ρ * grid[i] + step/2) / σ)
            elseif j == N
                Π[j, i] = 1 - cdf(d, (grid[j] - ρ * grid[i] - step/2) / σ)
            else
                Π[j, i] = cdf(d, (grid[j] - ρ * grid[i] + step/2) / σ) -
                         cdf(d, (grid[j] - ρ * grid[i] - step/2) / σ)
            end
        end
    end
    return collect(grid), Π
end

# Function to approximate the income process
function approx_income_process(N, ρ, σ, σ_α)
    # Use tauchen method to approximate the AR(1) component
    egrid, Π = tauchen(N, ρ, σ)
    agrid = [-sqrt(3/2)*σ_α, 0., sqrt(3/2)*σ_α]
    # Construct a larger grid of α .+ egrid for each α
    grid = [α + e for α in agrid for e in egrid]
    # Create a larger transition matrix where the
    # probability of moving from α to any other α' is zero
    Π = kron(I(3), Π)
    return grid, Π
end

Set up the income process using our estimated parameters:


In [ ]:
# Use estimated parameters from Question 3
N_AR1 = 5  # Grid points for AR(1) component
σ_ζ = sqrt(σ2_η)  # Innovation SD for AR(1)
σ_α_est = sqrt(σ2_α)  # SD of fixed effect

# Approximate the income process
egrid, Π = approx_income_process(N_AR1, ρ, σ_ζ, σ_α_est)

# Note: We ignore ε (measurement error) as it doesn't affect real income

println("Income process approximation:")
println("Number of states: ", length(egrid))
println("Grid range: [", round(minimum(egrid), digits=3), ", ",
        round(maximum(egrid), digits=3), "]")

Set up the model:


In [ ]:
# Model parameters
T = 40  # Working life
age_start = 25

# Create income grid with age-varying mean
K_y = length(egrid)
income_grid = zeros(K_y, T)
for t in 1:T
    for i in 1:K_y
        income_grid[i, t] = μ_t[t] + egrid[i]
    end
end

# Solve the model
function solve_model(p)
    (;T, Π, asset_grid) = p
    K_a = length(asset_grid)
    K_y = size(Π, 1)
    V = zeros(K_y, K_a, T+1)
    A = zeros(K_y, K_a, T)
    for t in reverse(1:T)
        iterate_value_function!(p, V, A, t)
    end
    return (;V, A)
end

function iterate_value_function!(p, V, A, t)
    for ai in axes(V, 2), yi in axes(V, 1)
        v, a_next = solve_value(ai, yi, V, p, t)
        V[yi, ai, t] = v
        A[yi, ai, t] = a_next
    end
end

function solve_value(ai, yi, V, p, t)
    (;σ, β, Π, r, asset_grid, income_grid, net_income) = p
    a = asset_grid[ai]
    y = exp(income_grid[yi, t])
    y_net = net_income(y)
    vmax = -Inf
    amax = 0
    for ai_next in eachindex(asset_grid)
        a_next = asset_grid[ai_next]
        c = y_net + a - a_next/(1+r)
        if c > 0
            @views v = utility(c, σ) + β * dot(Π[:, yi], V[:, ai_next, t+1])
            if v > vmax
                vmax = v
                amax = a_next
            end
        end
    end
    return vmax, amax
end

# Tax functions
no_tax(x) = x
progressive_tax(x, λ, τ) = λ * x^(1-τ)

# Model parameters
p = (;
    T,
    asset_grid = LinRange(0, 200, 100),
    income_grid,
    β = 0.96,
    r = 0.04,
    Π,
    σ = 2.,
    net_income = no_tax
)

Solve for the progressive tax parameter:


In [ ]:
function solve_steady_state(Π)
    K = size(Π, 2)
    vals, e = eigen(Π)
    return e[:, K] ./ sum(e[:, K])
end

function solve_lambda(τ, ygrid, pi_star)
    return dot(pi_star, ygrid) / dot(pi_star, ygrid.^(1-τ))
end

# Solve for stationary distribution
pi_star = solve_steady_state(p.Π)

# Choose progressivity
τ = 0.2

# Solve for λ
λ = solve_lambda(τ, exp.(income_grid[:, 1]), pi_star)

println("\nProgressive tax parameters:")
println("τ = ", τ)
println("λ = ", round(λ, digits=3))

Solve both models:


In [ ]:
println("\nSolving model without taxes...")
model0 = solve_model(p)

p_tax = (;p..., net_income = x -> progressive_tax(x, λ, τ))

println("Solving model with progressive taxes...")
model1 = solve_model(p_tax)

println("Done!")

Welfare decomposition:


In [ ]:
(;σ, β) = p

# Calculate average income
C = dot(pi_star, exp.(p.income_grid[:, 1]))

# Certainty equivalents by state
cbar0 = ((1-σ) * (1-β) / (1-β^T) * model0.V[:, 1, 1]).^(1/(1-σ))
cbar1 = ((1-σ) * (1-β) / (1-β^T) * model1.V[:, 1, 1]).^(1/(1-σ))

# Average certainty equivalents
Cbar0 = dot(pi_star, cbar0)
Cbar1 = dot(pi_star, cbar1)

# Aggregate welfare
W0 = dot(pi_star, model0.V[:, 1, 1])
W1 = dot(pi_star, model1.V[:, 1, 1])

# Value under perfect insurance
V_cert(x) = (1-β^T) / (1-β) * utility(x, σ)

# Decomposition
ω = (W1/W0)^(1/(1-σ)) - 1
γ = (V_cert(Cbar1) / V_cert(Cbar0))^(1/(1-σ)) - 1
α = ((W1 / V_cert(Cbar1)) / (W0 / V_cert(Cbar0)))^(1/(1-σ)) - 1

println("\n" * "="^60)
println("WELFARE DECOMPOSITION WITH NEW INCOME PROCESS")
println("="^60)
println("Total welfare gain (ω):           ", round(ω*100, digits=2), "%")
println("Insurance component (γ):          ", round(γ*100, digits=2), "%")
println("Redistribution component (α):     ", round(α*100, digits=2), "%")
println("\nVerification: (1+γ)(1+α) = ", round((1+γ)*(1+α), digits=4),
        " ≈ ", round(1+ω, digits=4))

# Display as table
results_welfare = DataFrame(
    Component = ["Total (1+ω)", "Insurance (1+γ)", "Redistribution (1+α)"],
    Value = [1+ω, 1+γ, 1+α],
    Percentage_gain = [ω*100, γ*100, α*100]
)

**Comparison with Recitation 4:**

In Recitation 4 we estimated a consumption equivalent welfare gain of 28% that was largely due to insurance (26%). Here we are seeing a 17% consumption equivalent increase in welfare that is largely attributable to redistribution (10%). This makes sense: in the previous case we did not allow for permanent differences among individuals, and so progressive taxation provided mostly insurance against less persistent shocks to income.